In [15]:
from elasticsearch import Elasticsearch
from datasets import load_from_disk

In [16]:
es = Elasticsearch('http://localhost:30001')

In [17]:
es.info()

{'name': '_eN3Dqz',
 'cluster_name': 'elasticsearch',
 'cluster_uuid': 'vExZ3cZVR7Gms-wk7OVygA',
 'version': {'number': '5.4.3',
  'build_hash': 'eed30a8',
  'build_date': '2017-06-22T00:34:03.743Z',
  'build_snapshot': False,
  'lucene_version': '6.5.1'},
 'tagline': 'You Know, for Search'}

In [18]:
!curl -XGET localhost:30001

{
  "name" : "_eN3Dqz",
  "cluster_name" : "elasticsearch",
  "cluster_uuid" : "vExZ3cZVR7Gms-wk7OVygA",
  "version" : {
    "number" : "5.4.3",
    "build_hash" : "eed30a8",
    "build_date" : "2017-06-22T00:34:03.743Z",
    "build_snapshot" : false,
    "lucene_version" : "6.5.1"
  },
  "tagline" : "You Know, for Search"
}


In [24]:
INDEX = 'wiki'
es.indices.create(index=INDEX)

{'acknowledged': True, 'shards_acknowledged': True}

In [25]:
!curl -XGET localhost:30001/_cat/indices?v

health status index uuid                   pri rep docs.count docs.deleted store.size pri.store.size
yellow open   wiki  5PqlAq6LRYq8hvTJMbnJ8w   5   1          0            0       650b           650b


In [31]:
import json
with open("/opt/ml/input/data/wikipedia_documents.json", "r", encoding="utf-8") as json_file:
    wiki_data = json.load(json_file)

In [33]:
contexts = list(
    dict.fromkeys([v["text"] for v in wiki_data.values()])
)  # set 은 매번 순서가 바뀌므로
print(f"Lengths of unique contexts : {len(contexts)}")

Lengths of unique contexts : 56737


In [35]:
for idx, text in enumerate(contexts):
    body = {'text': text}
    es.index(index=INDEX, doc_type='news', id=idx+1, body=body)

In [36]:
es.get(index=INDEX, doc_type='news', id=100)

{'_index': 'wiki',
 '_type': 'news',
 '_id': '100',
 '_version': 1,
 'found': True,
 '_source': {'text': '보통 오른손잡이는 반대쪽 왼손으로 넥 부분을 잡고 현을 누르며 오른손으로 현을 탄현한다. 그러나 반드시 이러한 자세를 갖출 필요는 없다. 왼손잡이인 사람은 가격, 상품 수 등이 크게 불리하기 때문에 왼손잡이용 기타를 준비할 수 없다는 것이 단점이지만, 오른손잡이에 맞춰진 기타에 기존의 반대 자세를 사용함으로써 자연스럽게 적응할 수는 있지만, 전기 컨트롤러의 위치가 다를 수 있다.\n\n현을 누르는 손은 넥 부분의 아래쪽을 빠져나가듯이 하여 엄지를 목 뒤쪽에 잡고, 집게손가락부터 새끼손가락까지 4개의 현을 손가락으로 누르는 것이 클래식 스타일, 곧 기본 자세이다. 또 엄지를 넥 위쪽 핑거보드에 가깝게 목 부분을 잡는 스타일도 주로 록 음악에서 이용된다. 시각이나 신체에 장애가 있는 사람은 목의 아래쪽부터 손가락을 위치해 현을 누르지 않고, 위쪽부터 손가락을 감싸듯이 하여 현을 눌러 연주하는 연주인도 있다.\n\n클래식 기타의 경우 왼발을 발판에 실어 다리의 위치를 높여서 몸체 측면의 움푹한 곳을 왼발의 넓적다리에 실어 기타를 몸 전체로 감싸 안는 자세로 연주한다. 그 밖의 통기타나 전기 기타의 경우는 발판은 쓰지 않고 기타 측면의 움푹한 곳을 오른쪽 다리의 넓적다리에 실어 연주하는 경우가 많다.\n\n또, 무대에서 클래식 기타는 반드시 의자에 앉아 연주하며, 그 밖에는 주로 기타 끈을 매고 어깨에 내려서 연주하는 경우가 많다.\n\n재즈 기타리스트는 앉아서 연주하는 경우가 많았다. 이는 한 때 빅 밴드 시대의 무대 형태의 흔적이라고 여겨진다. 록 음악인은 앉아서 연주하는 일은 많지 않았다.'}}

In [37]:
test = load_from_disk("/opt/ml/input/data/test_dataset")
test['validation']

Dataset({
    features: ['id', 'question'],
    num_rows: 600
})

In [45]:
body = {
		'query': {
				'match': {
						'text': '[Question]'
				}
		}
}
res = es.search(index=INDEX, body=body)